# Sebastian Schwab
> EE 399
>
> 5/15/23

Task 5 - https://github.com/sebschwab/ML-Training/blob/main/project%205/README.md

In [68]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import integrate
from mpl_toolkits.mplot3d import Axes3D

rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [12, 12]

dt = 0.01
T = 8
t = np.arange(0,T+dt,dt)
beta = 8/3
sigma = 10
rho = 28


nn_input = np.zeros((100*(len(t)-1),3))
nn_output = np.zeros_like(nn_input)



def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
    x, y, z = x_y_z
    return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

np.random.seed(123)
x0 = -15 + 30 * np.random.random((100, 3))

x_t = np.asarray([integrate.odeint(lorenz_deriv, x0_j, t)
                  for x0_j in x0])


In [69]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# FFNN


# Define activation functions
def logsig(x):
    return 1 / (1 + torch.exp(-x))

def radbas(x):
    return torch.exp(-torch.pow(x, 2))

def purelin(x):
    return x

# Define the model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(in_features=3, out_features=10)
        self.fc2 = nn.Linear(in_features=10, out_features=10)
        self.fc3 = nn.Linear(in_features=10, out_features=3)
        
    def forward(self, x):
        x = logsig(self.fc1(x))
        x = radbas(self.fc2(x))
        x = purelin(self.fc3(x))
        return x

# Define device: use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create model instance and move to device
model = MyModel().to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Convert numpy arrays to PyTorch tensors and move to device
nn_input = torch.from_numpy(nn_input).float().to(device)
nn_output = torch.from_numpy(nn_output).float().to(device)

# Train the model
for epoch in range(30):
    optimizer.zero_grad()
    outputs = model(nn_input)
    loss = criterion(outputs, nn_output)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, loss={loss.item():.4f}")


Epoch 1, loss=0.5223
Epoch 2, loss=0.4653
Epoch 3, loss=0.3693
Epoch 4, loss=0.2577
Epoch 5, loss=0.1526
Epoch 6, loss=0.0705
Epoch 7, loss=0.0196
Epoch 8, loss=0.0007
Epoch 9, loss=0.0081
Epoch 10, loss=0.0327
Epoch 11, loss=0.0642
Epoch 12, loss=0.0935
Epoch 13, loss=0.1138
Epoch 14, loss=0.1216
Epoch 15, loss=0.1168
Epoch 16, loss=0.1015
Epoch 17, loss=0.0797
Epoch 18, loss=0.0556
Epoch 19, loss=0.0334
Epoch 20, loss=0.0160
Epoch 21, loss=0.0050
Epoch 22, loss=0.0005
Epoch 23, loss=0.0015
Epoch 24, loss=0.0064
Epoch 25, loss=0.0129
Epoch 26, loss=0.0193
Epoch 27, loss=0.0240
Epoch 28, loss=0.0262
Epoch 29, loss=0.0257
Epoch 30, loss=0.0229


In [40]:
# LSTM MODEL

# Define the model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size=3, hidden_size=10, num_layers=1, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=10, hidden_size=10, num_layers=1, batch_first=True)
        self.fc = nn.Linear(in_features=10, out_features=3)
        
    def forward(self, x):
        x, _ = self.lstm1(x)
        x = logsig(x)
        x, _ = self.lstm2(x)
        x = radbas(x)
        x = self.fc(x)
        x = purelin(x)
        return x

# Define device: use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create model instance and move to device
model = MyModel().to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Convert numpy arrays to PyTorch tensors and move to device
# Assume nn_input shape (batch_size, sequence_length, input_size)
# Assume nn_output shape (batch_size, sequence_length, output_size)
# nn_input = torch.from_numpy(nn_input).float().to(device)
# nn_output = torch.from_numpy(nn_output).float().to(device)

# Train the model
for epoch in range(30):
    optimizer.zero_grad()
    outputs = model(nn_input)
    loss = criterion(outputs, nn_output)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, loss={loss.item():.4f}")

Epoch 1, loss=271.0275
Epoch 2, loss=245.1183
Epoch 3, loss=202.1373
Epoch 4, loss=153.3839
Epoch 5, loss=110.2976
Epoch 6, loss=82.3356
Epoch 7, loss=72.6105
Epoch 8, loss=78.2101
Epoch 9, loss=92.4139
Epoch 10, loss=106.3552
Epoch 11, loss=111.8972
Epoch 12, loss=106.2711
Epoch 13, loss=93.1268
Epoch 14, loss=80.0076
Epoch 15, loss=74.3785
Epoch 16, loss=72.7446
Epoch 17, loss=72.7576
Epoch 18, loss=73.4211
Epoch 19, loss=74.1224
Epoch 20, loss=74.5409
Epoch 21, loss=74.5858
Epoch 22, loss=74.3071
Epoch 23, loss=73.8289
Epoch 24, loss=73.3040
Epoch 25, loss=72.8710
Epoch 26, loss=72.6107
Epoch 27, loss=72.5280
Epoch 28, loss=72.5739
Epoch 29, loss=72.6844
Epoch 30, loss=72.8049


In [23]:
# RNN Model

# Define the model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.rnn = nn.RNN(input_size=3, hidden_size=10, num_layers=1, batch_first=True)
        self.fc = nn.Linear(in_features=10, out_features=3)
        
    def forward(self, x):
        x, _ = self.rnn(x)
        x = logsig(x)
        x = self.fc(x)
        x = purelin(x)
        return x

# Create model instance and move to device
model = MyModel().to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Convert numpy arrays to PyTorch tensors and move to device
# nn_input = torch.from_numpy(nn_input).float().to(device)
# nn_output = torch.from_numpy(nn_output).float().to(device)

# Convert to TensorDataset and DataLoader for batching
dataset = torch.utils.data.TensorDataset(nn_input, nn_output)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

# Train the model
for epoch in range(30):
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, loss={loss.item():.4f}")

Epoch 1, loss=6.0069
Epoch 2, loss=8.0741
Epoch 3, loss=5.0918
Epoch 4, loss=6.5783
Epoch 5, loss=1.0586
Epoch 6, loss=2.3336
Epoch 7, loss=0.5101
Epoch 8, loss=0.8545
Epoch 9, loss=2.6115
Epoch 10, loss=0.8453
Epoch 11, loss=0.3018
Epoch 12, loss=1.5362
Epoch 13, loss=0.7845
Epoch 14, loss=0.7159
Epoch 15, loss=0.3430
Epoch 16, loss=0.3797
Epoch 17, loss=0.2735
Epoch 18, loss=0.1417
Epoch 19, loss=0.1496
Epoch 20, loss=0.2604
Epoch 21, loss=0.1742
Epoch 22, loss=0.6531
Epoch 23, loss=0.1551
Epoch 24, loss=0.1305
Epoch 25, loss=0.2905
Epoch 26, loss=1.0716
Epoch 27, loss=0.2765
Epoch 28, loss=0.2152
Epoch 29, loss=0.1061
Epoch 30, loss=0.1669


In [47]:
import torch
import torch.nn as nn
# Echo State Network
class ESN1(nn.Module):
    def __init__(self, input_dim, reservoir_dim, output_dim):
        super(ESN1, self).__init__()
        self.input_dim = input_dim
        self.reservoir_dim = reservoir_dim
        self.output_dim = output_dim

        self.Win = nn.Parameter(torch.randn(reservoir_dim, input_dim) / np.sqrt(input_dim), requires_grad=False)
        self.W = nn.Parameter(torch.randn(reservoir_dim, reservoir_dim) / np.sqrt(reservoir_dim), requires_grad=True)
        self.Wout = nn.Parameter(torch.zeros(output_dim, reservoir_dim), requires_grad=True)

        # Optional: initialize W with spectral radius < 1 for stability
#         spectral_radius = torch.max(torch.abs(torch.eig(self.W)[0]))
#         self.W.data = self.W.data / spectral_radius

    def forward(self, x):
        batch_size, sequence_length, _ = x.size()

        reservoir_state = torch.zeros(batch_size, self.reservoir_dim, device=x.device, dtype=x.dtype)
        for t in range(sequence_length):
            reservoir_state = torch.tanh(torch.mm(x[:, t, :], self.Win.T) + torch.mm(reservoir_state, self.W.T))
        
        out = torch.mm(reservoir_state, self.Wout.T)
        return out

# Usage
input_dim = nn_input.shape[1]
reservoir_dim = 100  # Define as per your need
output_dim = nn_output.shape[1]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ESN1(input_dim, reservoir_dim, output_dim).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

# Convert numpy arrays to PyTorch tensors and move to device
# nn_input = torch.from_numpy(nn_input).float().to(device)
# nn_output = torch.from_numpy(nn_output).float().to(device)

# Train the model
for epoch in range(30):
    optimizer.zero_grad()
    outputs = model(nn_input.unsqueeze(0))
    loss = criterion(outputs, nn_output)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, loss={loss.item():.4f}")


Epoch 1, loss=279.7928
Epoch 2, loss=278.1538
Epoch 3, loss=276.5240
Epoch 4, loss=274.9017
Epoch 5, loss=273.2831
Epoch 6, loss=271.6636
Epoch 7, loss=270.0413
Epoch 8, loss=268.4180
Epoch 9, loss=266.7971
Epoch 10, loss=265.1818
Epoch 11, loss=263.5742
Epoch 12, loss=261.9755
Epoch 13, loss=260.3858
Epoch 14, loss=258.8045
Epoch 15, loss=257.2308
Epoch 16, loss=255.6646
Epoch 17, loss=254.1062
Epoch 18, loss=252.5561
Epoch 19, loss=251.0151
Epoch 20, loss=249.4837
Epoch 21, loss=247.9619
Epoch 22, loss=246.4495
Epoch 23, loss=244.9462
Epoch 24, loss=243.4514
Epoch 25, loss=241.9652
Epoch 26, loss=240.4876
Epoch 27, loss=239.0188
Epoch 28, loss=237.5592
Epoch 29, loss=236.1090
Epoch 30, loss=234.6680
